In [1]:
import os
os.chdir('../')
!pwd

/home/lok/Documents/ML_Projects/rice_leaf_disease


In [2]:
from pathlib import Path
from dataclasses import dataclass


@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [4]:
from rice_leaf_disease.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from rice_leaf_disease.utils.common import create_directories, read_yaml


class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])
        
    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks
        model_checkpoint_dir = os.path.dirname(config.checkpoint_model_filepath)
        
        create_directories([Path(model_checkpoint_dir),
                            Path(config.tensorboard_root_log_dir)])
        
        prepare_callback_config = PrepareCallbackConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)    
        )
        
        return prepare_callback_config

In [ ]:
import time
import tensorflow as tf

class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config
        
    
    """
    The @property decorator is used to define a method as a getter property.
    It allows to access the method as if it were an attribute, 
    without explicitly calling it like a function.    
    """    
    @property
    def _create_tb_callbacks(self):
        """
        Logs training metrics for visualization in TensorBoard.
        """
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}"
        )
        
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    
    @property
    def _create_checkpoint_callbacks(self):
        """
        Saves model checkpoints during training, ensuring best performing
        model is retained.
        """
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    
    def get_tb_checkpoint_callback(self):
        return [
            self._create_tb_callbacks,
            self._create_checkpoint_callbacks
        ]

In [10]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_checkpoint_callback()
except Exception as e:
    raise e

[2024-12-25 10:19:14,672: INFO : common : YAML file: config/config.yaml loaded successfully!]
[2024-12-25 10:19:14,691: INFO : common : YAML file: params.yaml loaded successfully!]
[2024-12-25 10:19:14,696: INFO : common : created directory at: artifacts]
[2024-12-25 10:19:14,699: INFO : common : created directory at: artifacts/prepare_callbacks/checkpoint_dir]
[2024-12-25 10:19:14,701: INFO : common : created directory at: artifacts/prepare_callbacks/tensorboard_log_dir]
